In [ ]:
# Bundles 2024 primary results for display in 2024 MTFP election guide
import pandas as pd

In [ ]:
xl = pd.ExcelFile('./inputs/sos-data/2024/primary/2024_06_11_Legislative Results.xlsx')
sheets = xl.sheet_names

output = []
for sheet in sheets:
    dfi = xl.parse(sheet)
    reporting = dfi.iloc[:,0][1]
    time = dfi.iloc[:,0][2]
    district = dfi.iloc[:,0][5].replace('STATE SENATOR','S').replace('STATE REPRESENTATIVE','H').replace(' DISTRICT','D').replace(' ','-')

    data = dfi[5:].drop(dfi.columns[0], axis='columns').transpose()
    data.columns = data.iloc[0]
    data.drop(data.index[0], inplace=True)
    data.reset_index(inplace=True)
    output.append(list(data.apply(lambda row: ({
        'district': district,
        'party': row['County'].split('\n')[1],
        'candidate': row['County'].split('\n')[0],
        'votes': row['TOTALS'],
    }), axis=1)))
    
output


[[{'district': 'SD-2',
   'party': 'Republican',
   'candidate': 'DOUG ADAMS',
   'votes': 2318}],
 [{'district': 'SD-2',
   'party': 'Democrat',
   'candidate': 'DAVE FERN',
   'votes': 2344}],
 [{'district': 'SD-3',
   'party': 'Republican',
   'candidate': 'CARL GLIMM',
   'votes': 3802}],
 [{'district': 'SD-3',
   'party': 'Democrat',
   'candidate': 'ANGELA KENNEDY',
   'votes': 1394}],
 [{'district': 'SD-5',
   'party': 'Republican',
   'candidate': 'MARQUIS L LAUDE',
   'votes': 1427},
  {'district': 'SD-5',
   'party': 'Republican',
   'candidate': 'MATT REGIER',
   'votes': 2890}],
 [{'district': 'SD-5',
   'party': 'Democrat',
   'candidate': 'LINK NEIMARK',
   'votes': 1015}],
 [{'district': 'SD-7',
   'party': 'Republican',
   'candidate': 'GREG HERTZ',
   'votes': 4409}],
 [{'district': 'SD-7',
   'party': 'Democrat',
   'candidate': 'CRAIG PABLO',
   'votes': 1573}],
 [{'district': 'SD-13',
   'party': 'Republican',
   'candidate': 'JOSH KASSMIER',
   'votes': 3321},
  {'

In [3]:
tall = pd.DataFrame([item for district in output for item in district])

In [4]:
tall

,district,party,candidate,votes
0,SD-2,Republican,DOUG ADAMS,2318
1,SD-2,Democrat,DAVE FERN,2344
2,SD-3,Republican,CARL GLIMM,3802
3,SD-3,Democrat,ANGELA KENNEDY,1394
4,SD-5,Republican,MARQUIS L LAUDE,1427
...,...,...,...,...
297,HD-99,Republican,RYAN DARLING,1017
298,HD-99,Democrat,TOM FRANCE,1536
299,HD-100,Republican,CHRISTOPHER PAUL BUCKLES,353
300,HD-100,Democrat,SJ HOWELL,1230


In [5]:
def summarize(row):
    total_votes = row['votes'].sum()
    gop_votes = row[row['party'] == 'Republican']['votes'].sum()
    dem_votes = row[row['party'] == 'Democrat']['votes'].sum()
    return pd.Series({
        'total_primary_votes': total_votes,
        'fraction_primary_votes_to_gop': gop_votes / total_votes,
        'gop_primary_votes': gop_votes,
        'gop_candidates': ', '.join(row[row['party'] == 'Republican']['candidate']),
        'dem_primary_votes': dem_votes,
        'dem_candidates': ', '.join(row[row['party'] == 'Democrat']['candidate']),
        
    }, index=['total_primary_votes','fraction_primary_votes_to_gop','gop_primary_votes','gop_candidates','dem_primary_votes','dem_candidates'])

summary = tall.groupby(['district']).apply(summarize).sort_values('fraction_primary_votes_to_gop', ascending=False)

In [6]:
# Merge in Daves districting scores
daves_hd = pd.read_csv('../inputs/daves-redistricting/mt-house-district-statistics.csv',index_col=False)
daves_hd['daves_score_r_plus'] = round(100*(daves_hd['Rep'] - daves_hd['Dem']))
daves_hd['district'] = 'HD-' + daves_hd['ID']

daves_sd = pd.read_csv('../inputs/daves-redistricting/mt-senate-district-statistics.csv',index_col=False)
daves_sd['daves_score_r_plus'] = round(100*(daves_sd['Rep'] - daves_sd['Dem']))
daves_sd['district'] = 'SD-' + daves_sd['ID']

daves = daves_hd.append(daves_sd)
daves


/var/folders/gk/_q4g8knj7354h1qcc0gsdcq00000gp/T/ipykernel_4821/2078633442.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  daves = daves_hd.append(daves_sd)


,ID,Total Pop,Deviation,Dem,Rep,Oth,Total VAP,White,Minority,Hispanic,Black,Asian,Native,Pacific,daves_score_r_plus,district
0,Un,0,0.0000,0.0000,0.0000,0.0000,0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,HD-Un
1,1,10823,-0.0004,0.2533,0.7303,0.0165,8854,0.9086,0.0914,0.0270,0.0049,0.0089,0.0374,0.0029,48.0,HD-1
2,2,10904,0.0071,0.2565,0.7236,0.0199,8744,0.9145,0.0855,0.0183,0.0045,0.0091,0.0396,0.0027,47.0,HD-2
3,3,10764,-0.0058,0.5301,0.4500,0.0198,8873,0.9167,0.0833,0.0275,0.0036,0.0132,0.0267,0.0021,-8.0,HD-3
4,4,10774,-0.0049,0.4747,0.5022,0.0231,8472,0.9055,0.0945,0.0238,0.0057,0.0111,0.0360,0.0035,3.0,HD-4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47,47,21728,0.0034,0.6112,0.3683,0.0204,16843,0.8748,0.1252,0.0499,0.0081,0.0234,0.0373,0.0026,-24.0,SD-47
48,48,21513,-0.0065,0.7127,0.2612,0.0261,18039,0.8170,0.1830,0.0971,0.0121,0.0206,0.0466,0.0024,-45.0,SD-48
49,49,21621,-0.0015,0.5886,0.3815,0.0299,17340,0.8600,0.1400,0.0370,0.0128,0.0221,0.0585,0.0026,-21.0,SD-49
50,50,21473,-0.0084,0.6649,0.3069,0.0282,17611,0.8659,0.1341,0.0393,0.0135,0.0217,0.0485,0.0038,-36.0,SD-50


In [7]:
summary = summary.merge(daves[['district','daves_score_r_plus', 'Native']], left_on='district', right_on='district')

In [8]:
summary.to_csv('../outputs/2024-legislative-primaries-by-party-vote.csv', index=False)
